In [1]:
import glob
import re
import spacy
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
import numpy as np

x_train = []
y_train = []

x_test = []
y_test = []
y_test_id = []

y_train_cc = []
y_train_cd = [] #seperate for the beggining beacuse the files will be seperate

x_train_cc_ids = []
x_train_cd_ids = []

In [4]:
data = {'ID': [],
        'Difficulties' : [],
        'uh': [],
        'retrace_no_corr': [],
        'retrace_with_corr': [],
        'interruption': [],
        'gfi': [],
        # 'ari': [],
        'unique_words': [],
        'text_len': [],
        # 'bag_of_words': [],
        'prediction' : []}

Δημιουργία χαρακτηριστικών

-------------------------------------

Functions to create characteristics

In [5]:
def calculate_ari(sentences_from_file):
  all_sentences = find_all_sentences(sentences_from_file)
  total_characters = 0
  total_words = 0
  total_sentences = len(all_sentences)
  for s in all_sentences:
    for word in s:
      total_words += 1
      total_characters += len(word)

  ari = 4.71 * (total_characters / (total_words * 1.0)) + 0.5 * (total_words / (total_sentences * 1.0)) - 21.43
  return ari

In [6]:
def find_all_sentences (sentences_from_file):
  regex = "[^a-zA-Z.?!']+"
  p=re.compile(regex)
  all_sentences = []
  sentence = []
  for s in sentences_from_file:
    s_split = s.split()
    for word in s_split:
      if (not re.search(p, word)):
        if word != "." and word != "!" and word != "?":
          sentence.append(word)
        elif word == "." or word == "!" or word == "?":
          if len(sentence) != 0:
            all_sentences.append(sentence)
          sentence = []

  if len(sentence) != 0:
    all_sentences.append(sentence)

  return all_sentences


In [7]:
def calculate_gfi(sentences_from_file):
  hard_words = 0
  all_words = []
  all_sentences_actually = find_all_sentences(sentences_from_file)
  for s in all_sentences_actually:
    for word in s:
      all_words.append(word)
      if(len(word)>=7):
        hard_words += 1

  total_words = len(all_words)
  total_sentences = len(all_sentences_actually)
  avg_words_per_sentence = total_words / (total_sentences * 1.0)
  hard_words_percent = hard_words / (total_sentences * 1.0)
  gfi = 0.4 * (avg_words_per_sentence + 100 * hard_words_percent)
  return gfi

In [8]:
def find_unique_words(sentences_from_file):
  all_words = []
  for sentence in sentences_from_file:
    for word in sentence:
      all_words.append(word)

  unique_words = set(all_words)
  return len(set(all_words)) / (len(all_words) * 1.0)

In [9]:
def get_text_len(sentences_from_file):
  all_words = []
  for sentence in sentences_from_file:
    for word in sentence:
      all_words.append(word)

  return len(set(all_words))

Ανάγνωση των δεδομένων εκπαίδευσης και δημιουργία dataset. Κατά την διάρκεια της διαπέρασης των δεδομένων, υπολογίζονται και άλλα χαρακτηριστικά, όπως τα χαρακτηριστικά της κωδικοποίησης CHAT

---------------------------------------------------------------------------

Reading training data and creating the dataset. While reading the data, other characteristics are calculated, like characteristics based on CHAT encoding

In [10]:
path = glob.glob("/content/drive/MyDrive/ADReSS-IS2020-train/ADReSS-IS2020-data/train/transcription"+"/*/*.cha")
words_per_participant_cc = []
nlp = spacy.load('en_core_web_sm')

for filename in path:
  # print("-------------------------------------", filename, "-------------------------------------")
  train_file = open(filename, "r")
  print(filename)

  keep_filename = filename
  keep_filename = keep_filename.replace("/content/drive/MyDrive/ADReSS-IS2020-train/ADReSS-IS2020-data/train/transcription/cc/", "")
  keep_filename = keep_filename.replace("/content/drive/MyDrive/ADReSS-IS2020-train/ADReSS-IS2020-data/train/transcription/c/", "")
  keep_filename = keep_filename.replace(".cha", "")
  # print(keep_filename)
  x_train_cc_ids.append(keep_filename)

  train_text_non_AD = train_file.read()
  # x_train.append(train_text_non_AD)
  cc_or_cd = filename.replace("/content/drive/MyDrive/ADReSS-IS2020-train/ADReSS-IS2020-data/train/transcription/", "")
  if "cc" in cc_or_cd:
    y_train_cc.append(0)
  else :
    y_train_cc.append(1)


  sentences = train_text_non_AD.split("\n")
  regex = "[^a-zA-Z]+" #this is so that i can seperate the words and take out the symbols
  p=re.compile(regex)

  par_talk_sentences = []
  begin_answer = False
  for item in sentences:
    # print(item)
    if item.startswith('*PAR'):
      # print(item)
      par_talk_sentences.append(item)
      begin_answer = True #check if a *PAR has started in the previous line

    elif begin_answer == True:
      if not item.startswith('*') and not item.startswith('%') and not item.startswith('@'): #if it has started but has one of those symbol, it ends (else)
        # print(item)
        par_talk_sentences.append(item)
        # if(re.search(p, item)):
        #   print(item)
      else:
         begin_answer = False

    elif (begin_answer==False):
      pass

  talker_words = []
  difficulty_words = 0
  no_uhs = 0
  retrace_no_correction = 0
  retrace_with_correction = 0
  interruptions = 0
  sentence_word_count = 0
  gfi = calculate_gfi(par_talk_sentences)
  text_length = get_text_len(par_talk_sentences)
  # ari = calculate_ari(par_talk_sentences)
  unique = find_unique_words(par_talk_sentences)
  # print("gfi : ",  gfi)
  for item in par_talk_sentences:
    all_words = item.split()
    # print(all_words)
    for word in all_words:
      if(not re.search(p, word)):
        talker_words.append(word)
        # print(word)
      # print ("------------------------------------------------------------------------------------------------------------------------")
      # print(word, re.search(p, word))
      else:
        if "'" in word:
          new_word = word.replace("'", "")
          if(not re.search(p, new_word)):
            talker_words.append(new_word)
            # print(new_word)
        elif "(" in word:
          new_word = word.replace("(", "")
          new_word = new_word.replace(")", "")
          difficulty_words += 1
          if(not re.search(p, new_word)):
            talker_words.append(new_word)
        elif "&uh" in word:
          no_uhs += 1
          new_word = word.replace("&", "")
          if(not re.search(p, new_word)):
            talker_words.append(new_word)
        elif "[/]" in word:
          retrace_no_correction += 1
        elif "[//]" in word:
          retrace_with_correction += 1
        elif "+//" in word:
          interruptions += 1



    lemmatized_talker_words = []
    for word in talker_words:
      if word != "." and word != "?" and word != "!":
        lemmatized_talker_words.append(nlp(word)[0].lemma_)
      else:
        lemmatized_talker_words.append(word)
      # print(word, " : ", nlp(word)[0].lemma_)

  # calculate_gfi(par_talk_sentences)
  words_per_participant_cc.append(lemmatized_talker_words)

  # vectorizer = CountVectorizer(vocabulary=set(lemmatized_talker_words))
  # corpus = ['woman', 'window']
  # X = vectorizer.transform(corpus)
  # print(X.toarray())

  # print(lemmatized_talker_words)
  # print(" == ", y_train_cc[-1])
  # print("Difficculty", difficulty_words)
  data['ID'].append(keep_filename)
  data['Difficulties'].append(difficulty_words/(text_length * 1.0))
  # data['Difficulties'].append(difficulty_words)
  data['uh'].append(no_uhs)

  if "cc" in cc_or_cd:
    data['prediction'].append(0)
  else :
    data['prediction'].append(1)

  data['retrace_no_corr'].append(retrace_no_correction)
  data['retrace_with_corr'].append(retrace_with_correction)
  data['interruption'].append(interruptions)
  data['gfi'].append(gfi)
  # data['ari'].append(ari)
  data['unique_words'].append(unique)
  data['text_len'].append(text_length)
  # data['bag_of_words'].append(X.toarray())
  # data['age'].append(-1)



/content/drive/MyDrive/ADReSS-IS2020-train/ADReSS-IS2020-data/train/transcription/cd/S097.cha
/content/drive/MyDrive/ADReSS-IS2020-train/ADReSS-IS2020-data/train/transcription/cd/S080.cha
/content/drive/MyDrive/ADReSS-IS2020-train/ADReSS-IS2020-data/train/transcription/cd/S086.cha
/content/drive/MyDrive/ADReSS-IS2020-train/ADReSS-IS2020-data/train/transcription/cd/S110.cha
/content/drive/MyDrive/ADReSS-IS2020-train/ADReSS-IS2020-data/train/transcription/cd/S082.cha
/content/drive/MyDrive/ADReSS-IS2020-train/ADReSS-IS2020-data/train/transcription/cd/S087.cha
/content/drive/MyDrive/ADReSS-IS2020-train/ADReSS-IS2020-data/train/transcription/cd/S083.cha
/content/drive/MyDrive/ADReSS-IS2020-train/ADReSS-IS2020-data/train/transcription/cd/S116.cha
/content/drive/MyDrive/ADReSS-IS2020-train/ADReSS-IS2020-data/train/transcription/cd/S118.cha
/content/drive/MyDrive/ADReSS-IS2020-train/ADReSS-IS2020-data/train/transcription/cd/S089.cha
/content/drive/MyDrive/ADReSS-IS2020-train/ADReSS-IS2020-dat

In [11]:
import pandas as pd

print(data)

df = pd.DataFrame(data)
print(df.columns)
print(df)

{'ID': ['/content/drive/MyDrive/ADReSS-IS2020-train/ADReSS-IS2020-data/train/transcription/cd/S097', '/content/drive/MyDrive/ADReSS-IS2020-train/ADReSS-IS2020-data/train/transcription/cd/S080', '/content/drive/MyDrive/ADReSS-IS2020-train/ADReSS-IS2020-data/train/transcription/cd/S086', '/content/drive/MyDrive/ADReSS-IS2020-train/ADReSS-IS2020-data/train/transcription/cd/S110', '/content/drive/MyDrive/ADReSS-IS2020-train/ADReSS-IS2020-data/train/transcription/cd/S082', '/content/drive/MyDrive/ADReSS-IS2020-train/ADReSS-IS2020-data/train/transcription/cd/S087', '/content/drive/MyDrive/ADReSS-IS2020-train/ADReSS-IS2020-data/train/transcription/cd/S083', '/content/drive/MyDrive/ADReSS-IS2020-train/ADReSS-IS2020-data/train/transcription/cd/S116', '/content/drive/MyDrive/ADReSS-IS2020-train/ADReSS-IS2020-data/train/transcription/cd/S118', '/content/drive/MyDrive/ADReSS-IS2020-train/ADReSS-IS2020-data/train/transcription/cd/S089', '/content/drive/MyDrive/ADReSS-IS2020-train/ADReSS-IS2020-data

In [12]:
test_data = {'ID': [],
        'Difficulties' : [],
        'uh': [],
        'retrace_no_corr': [],
        'retrace_with_corr': [],
        'interruption': [],
        'gfi': [],
        # 'ari': [],
        'unique_words': [],
        'text_len': [],
        # 'bag_of_words': [],
        'prediction' : []}

- Ανάγνωση των δεδομένων ελέγχου και δημιουργία dataset. Κατά την διάρκεια της διαπέρασης των δεδομένων, υπολογίζονται και άλλα χαρακτηριστικά, όπως τα χαρακτηριστικά της κωδικοποίησης CHAT.
- Η μόνη διαφορά με τα δεδομένα εκπαίδευσης είναι ότι τα δεδομένα ελέγχου έχουν το prediction στο αρχείο labels.txt μαζί με κάποια άλλα χαρακτηριστικά, επομένως υπολογίζονται πρώτα τα χαρακτηριστικά και παρακάτω θα συνδεθούν με τα predictions

-----------------------------------------------------------------------

- Reading test data and creating the dataset. While reading the data, other characteristics are calculated, like characteristics based on CHAT encoding
- The only difference with training data, is that the predictions of the train data are saved on the file labels.txt along with some other data, so at first the characteristics are calculated and after they will be connected to the predictions  

In [13]:
#test data
path = glob.glob("/content/drive/MyDrive/ADReSS-IS2020-test/ADReSS-IS2020-data/test/transcription"+"/*.cha")
words_per_participant_test = []

for filename in path:
  # print("-------------------------------------", filename, "-------------------------------------")
  test_file = open(filename, "r")
  keep_filename = filename
  keep_filename = keep_filename.replace("/content/drive/MyDrive/ADReSS-IS2020-test/ADReSS-IS2020-data/test/transcription/", "")
  keep_filename = keep_filename.replace(".cha", "")
  # print(keep_filename)
  # print(filename)
  test_text_all = test_file.read()
  # print(test_text_all)

  sentences = test_text_all.split("\n")
  per_talk_sentences_test = []
  begin_answer = False
  for item in sentences:
    if item.startswith('*PAR'):
      per_talk_sentences_test.append(item)
      begin_answer = True

    elif begin_answer == True:
      if not item.startswith('*') and not item.startswith('%') and not item.startswith('@'):
        per_talk_sentences_test.append(item)

      else:
        begin_answer = False

    elif begin_answer == False:
      pass

  talker_words = []
  difficulty_words = 0
  no_uhs = 0
  retrace_no_correction = 0
  retrace_with_correction = 0
  interruptions = 0
  gfi = calculate_gfi(per_talk_sentences_test)
  # ari = calculate_ari(per_talk_sentences_test)
  unique = find_unique_words(par_talk_sentences)
  text_length = get_text_len(par_talk_sentences)
  # print("gfi : ",  gfi)
  for item in per_talk_sentences_test:
    all_words_test = item.split()

    for word in all_words_test:
      if not re.search(p, word):
        talker_words.append(word)

      else:
        if "'" in word:
          new_word = word.replace("'", "")
          if not re.search(p, new_word):
            talker_words.append(new_word)
        elif "(" in word:
          new_word = word.replace("(", "")
          new_word = new_word.replace(")", "")
          difficulty_words += 1
          if(not re.search(p, new_word)):
            talker_words.append(new_word)
        elif "&uh" in word:
          no_uhs += 1
          new_word = word.replace("&", "")
          if(not re.search(p, new_word)):
            talker_words.append(new_word)
        elif "[/]" in word:
          retrace_no_correction += 1
        elif "[//]" in word:
          retrace_with_correction += 1
        elif "+//" in word:
          interruptions += 1

    lemmatized_talker_words_test = []
    for word in talker_words:
      lemmatized_talker_words_test.append(nlp(word)[0].lemma_)


  test_data['ID'].append(keep_filename)
  test_data['Difficulties'].append(difficulty_words/(text_length * 1.0))
  # test_data['Difficulties'].append(difficulty_words)
  test_data['uh'].append(no_uhs)
  test_data['prediction'].append(-1)
  test_data['retrace_no_corr'].append(retrace_no_correction)
  test_data['retrace_with_corr'].append(retrace_with_correction)
  test_data['interruption'].append(interruptions)
  test_data['gfi'].append(gfi)
  # test_data['ari'].append(ari)
  test_data['unique_words'].append(unique)
  test_data['text_len'].append(text_length)
  # test_data['bag_of_words'].append(bag_of_words)
  # test_data['age'].append(-1)

  words_per_participant_test.append(lemmatized_talker_words_test)
  # print(lemmatized_talker_words_test)


In [14]:
print(test_data)

df_t = pd.DataFrame(test_data)
print(df_t.columns)
print(df_t)

{'ID': ['S172', 'S171', 'S174', 'S166', 'S160', 'S161', 'S167', 'S170', 'S168', 'S165', 'S173', 'S164', 'S169', 'S163', 'S162', 'S182', 'S180', 'S201', 'S179', 'S178', 'S189', 'S184', 'S175', 'S187', 'S188', 'S205', 'S198', 'S206', 'S191', 'S177', 'S197', 'S203', 'S204', 'S193', 'S194', 'S192', 'S196', 'S186', 'S202', 'S183', 'S195', 'S181', 'S185', 'S199', 'S200', 'S176', 'S190', 'S207'], 'Difficulties': [0.0, 0.06382978723404255, 0.06382978723404255, 0.0425531914893617, 0.0425531914893617, 0.10638297872340426, 0.3191489361702128, 0.0851063829787234, 0.1276595744680851, 0.1276595744680851, 0.0851063829787234, 0.06382978723404255, 0.0, 0.0851063829787234, 0.02127659574468085, 0.06382978723404255, 0.02127659574468085, 0.0, 0.06382978723404255, 0.10638297872340426, 0.06382978723404255, 0.02127659574468085, 0.3617021276595745, 0.425531914893617, 0.02127659574468085, 0.0, 0.0851063829787234, 0.0851063829787234, 0.19148936170212766, 0.0, 0.0425531914893617, 0.14893617021276595, 0.0, 0.02127

Διαβασμα predictions και συνδεση τους με τα προηγούμενα δεδομένα

---------------------------------------------------------------------------

Reading the predictions and connecting them to the previous data

In [15]:
filename = "/content/drive/MyDrive/ADReSS-IS2020-test/ADReSS-IS2020-data/test/labels.txt"
test_file = open(filename, "r")
# print(filename)
test_text_all = test_file.read()

index_id = -1
index_label = -1
age = -1

sentences = test_text_all.split("\n")
first_line_words = sentences[0].split(';')
i=0
for label in first_line_words:
  # print(label)
  label = label.replace(" ", "")
  # print(label)
  if label == "ID":
    index_id = i
    # print("id in label: ", i)
  elif label == "Label":
    index_label = i
    # print("label in label: ", i)
  elif label == "age":
    age = i

  i += 1

new_sentences = []
for sentence in sentences:
  new_sentence = []
  sentence = sentence.split(";")
  print(" s: ", sentence)
  for word in sentence:
    word = word.replace(" ", "")
    new_sentence.append(word)

  new_sentences.append(new_sentence)

i=0
y_test_id = []
ages = []
for sentence in new_sentences:
  # print ("sentence: ", sentence)

  if i != 0:
    y_test_id.append(sentence[index_id])
    y_test.append(sentence[index_label])
    ages.append(sentence[age])
  i+= 1

print("_________________________________________________")
print(y_test_id)
print(len(new_sentences))
print("len age: ", len(ages))
print(len(y_test_id))


i=0
for i in range(len(y_test_id)):
  # print("len: ", len(y_test_id), "==============================================")
  # print("id: ", y_test_id[i], " , label: ", y_test[i])
  list_obj = [y_test_id[i]]
  df_t.loc[df_t['ID'].isin(list_obj), 'prediction'] = y_test[i]



 s:  ['ID   ', ' age', ' gender', 'Label ', 'mmse']
 s:  ['S160 ', ' 63', ' 1 ', ' 0 ', ' 28']
 s:  ['S161 ', ' 55', ' 1 ', ' 0 ', ' 29']
 s:  ['S162 ', ' 67', ' 1 ', ' 1 ', ' 24']
 s:  ['S163 ', ' 71', ' 0 ', ' 0 ', ' 30']
 s:  ['S164 ', ' 73', ' 1 ', ' 1 ', ' 21']
 s:  ['S165 ', ' 64', ' 0 ', ' 1 ', ' 15']
 s:  ['S166 ', ' 73', ' 1 ', ' 0 ', ' 29']
 s:  ['S167 ', ' 65', ' 0 ', ' 1 ', ' 28']
 s:  ['S168 ', ' 71', ' 0 ', ' 1 ', ' 27']
 s:  ['S169 ', ' 78', ' 0 ', ' 1 ', ' 26']
 s:  ['S170 ', ' 51', ' 0 ', ' 0 ', ' 28']
 s:  ['S171 ', ' 69', ' 1 ', ' 1 ', ' 23']
 s:  ['S172 ', ' 78', ' 1 ', ' 0 ', ' 30']
 s:  ['S173 ', ' 79', ' 1 ', ' 1 ', ' 17']
 s:  ['S174 ', ' 67', ' 1 ', ' 0 ', ' 29']
 s:  ['S175 ', ' 69', ' 1 ', ' 0 ', ' 30']
 s:  ['S176 ', ' 69', ' 0 ', ' 1 ', ' 27']
 s:  ['S177 ', ' 65', ' 0 ', ' 0 ', ' 30']
 s:  ['S178 ', ' 68', ' 1 ', ' 0 ', ' 30']
 s:  ['S179 ', ' 74', ' 0 ', ' 1 ', ' 10']
 s:  ['S180 ', ' 74', ' 0 ', ' 0 ', ' 29']
 s:  ['S181 ', ' 65', ' 1 ', ' 1 ', ' 17']
 s

Αποθήκευση των dataset σε αρχεία csv
  
---------------------------------------------------------------------------

Saving the dataset in csv files

In [16]:
df.to_csv("train_data.csv", index = False)
df_t.to_csv("test_data.csv", index = False)